In [1]:
!python -m pip install --user --upgrade pip

In [110]:
# Ignore the warnings
import warnings
# warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# System related and data input controls
import os
import pickle
import tqdm

# Data manipulation and visualization
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_rows = 20
pd.options.display.max_columns = 20
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing

# Modeling algorithms
# General
import statsmodels.api as sm
from scipy import stats

# Model selection
from sklearn.model_selection import train_test_split

# Evaluation metrics
# for regression
from sklearn.metrics import mean_squared_log_error, mean_squared_error,  r2_score, mean_absolute_error

In [111]:
location = '/Users/johanjun/Documents/2021/seoul_bike_share/data/df_address_join.pkl'
with open(location, 'rb') as f:
    raw_all = pickle.load(f)
raw_all

,rent_date,rent_hour,station_no,station_name,rent_type,sex,age_type,rent_count,momentum,green,distance,use_hour,address,gu
0,2021-01-01,0.00,1347,1347. 길음래미안아파트817동 상가 앞,정기권,,~10대,2.00,28.90,0.23,992.78,13.00,서울특별시 성북구 길음로 33,성북구
1,2021-01-01,0.00,1347,1347. 길음래미안아파트817동 상가 앞,정기권,,20대,1.00,11.24,0.07,283.72,1.00,서울특별시 성북구 길음로 33,성북구
2,2021-01-01,2.00,1347,1347. 길음래미안아파트817동 상가 앞,일일권,,20대,1.00,42.24,0.43,"1,839.04",37.00,서울특별시 성북구 길음로 33,성북구
3,2021-01-01,2.00,1347,1347. 길음래미안아파트817동 상가 앞,일일권,M,20대,2.00,50.75,0.44,"1,912.72",73.00,서울특별시 성북구 길음로 33,성북구
4,2021-01-01,5.00,1347,1347. 길음래미안아파트817동 상가 앞,일일권,M,~10대,1.00,183.65,1.66,"7,134.79",47.00,서울특별시 성북구 길음로 33,성북구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
720131,NaN,NaN,3304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,남부순환로 1917,관악구
720132,NaN,NaN,3305,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,남부순환로 1939,관악구
720133,NaN,NaN,2263,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서울특별시 서초구 바우뫼로 43,서초구
720134,NaN,NaN,2543,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,서초구 동작대로 164,서초구


In [112]:
raw_all.describe(include='all')

,rent_date,rent_hour,station_no,station_name,rent_type,sex,age_type,rent_count,momentum,green,distance,use_hour,address,gu
count,720123,"720,123.00","720,136.00",720123,720123,720123,720123,"720,123.00","720,123.00","720,123.00","720,123.00","720,123.00",717603,717603
unique,30,NaN,NaN,2188,4,5,7,NaN,NaN,NaN,NaN,NaN,2068,25
top,2021-01-25,NaN,NaN,583. 청계천 생태교실 앞,정기권,,20대,NaN,NaN,NaN,NaN,NaN,서울특별시 성동구 마장로39길 51,강서구
freq,47402,NaN,NaN,2532,585993,318154,270494,NaN,NaN,NaN,NaN,NaN,2532,65702
mean,NaN,14.20,"1,445.81",NaN,NaN,NaN,NaN,1.09,81.75,0.71,"3,042.29",28.11,NaN,NaN
std,NaN,5.24,"1,015.02",NaN,NaN,NaN,NaN,0.34,117.69,1.00,"4,291.63",33.28,NaN,NaN
min,NaN,0.00,3.00,NaN,NaN,NaN,NaN,1.00,0.00,0.00,0.00,0.00,NaN,NaN
25%,NaN,11.00,593.00,NaN,NaN,NaN,NaN,1.00,20.73,0.18,789.60,8.00,NaN,NaN
50%,NaN,15.00,"1,228.00",NaN,NaN,NaN,NaN,1.00,44.75,0.39,"1,682.76",16.00,NaN,NaN
75%,NaN,18.00,"2,113.00",NaN,NaN,NaN,NaN,1.00,95.76,0.83,"3,580.00",37.00,NaN,NaN


In [113]:
raw_all = raw_all.dropna(subset=['rent_hour'])

In [114]:
raw_all.shape

(720123, 14)

In [115]:
raw_all['rent_hour'] = raw_all['rent_hour'].apply(lambda x : int(x))

In [116]:
if 'rent_date' in raw_all.columns:
    raw_all['rent_date'] = pd.to_datetime(raw_all['rent_date'])
    raw_all['DateTime'] = pd.to_datetime(raw_all['rent_date'])
raw_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 720123 entries, 0 to 720122
Data columns (total 15 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   rent_date     720123 non-null  datetime64[ns]
 1   rent_hour     720123 non-null  int64         
 2   station_no    720123 non-null  int64         
 3   station_name  720123 non-null  object        
 4   rent_type     720123 non-null  object        
 5   sex           720123 non-null  object        
 6   age_type      720123 non-null  object        
 7   rent_count    720123 non-null  float64       
 8   momentum      720123 non-null  float64       
 9   green         720123 non-null  float64       
 10  distance      720123 non-null  float64       
 11  use_hour      720123 non-null  float64       
 12  address       717590 non-null  object        
 13  gu            717590 non-null  object        
 14  DateTime      720123 non-null  datetime64[ns]
dtypes: datetime64[ns]

In [121]:
for i, v in enumerate(raw_all['rent_date']):
    raw_all['rent_date'][i] = raw_all['rent_date'][i].replace(hour=raw_all['rent_hour'][i])
    if i % 10000 == 0:
        print(i)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000


In [122]:
raw_all.tail()

,rent_date,rent_hour,station_no,station_name,rent_type,sex,age_type,rent_count,momentum,green,distance,use_hour,address,gu,DateTime
720118,2021-01-25 19:00:00,19,3878,3878. 장로회신학대학교 주기철 기념관,일일권,,20대,1.00,76.28,0.69,"2,963.57",53.00,NaN,NaN,2021-01-25
720119,2021-01-25 19:00:00,19,3878,3878. 장로회신학대학교 주기철 기념관,일일권,F,20대,2.00,105.18,0.99,"4,278.46",107.00,NaN,NaN,2021-01-25
720120,2021-01-27 13:00:00,13,3878,3878. 장로회신학대학교 주기철 기념관,정기권,,20대,1.00,73.45,0.51,"2,182.24",13.00,NaN,NaN,2021-01-27
720121,2021-01-30 16:00:00,16,3878,3878. 장로회신학대학교 주기철 기념관,일일권,,20대,1.00,49.69,0.49,"2,091.34",16.00,NaN,NaN,2021-01-30
720122,2021-01-31 18:00:00,18,3878,3878. 장로회신학대학교 주기철 기념관,일일권,M,20대,1.00,70.66,0.69,"2,973.84",20.00,NaN,NaN,2021-01-31


In [128]:
raw_all.set_index('rent_date', inplace=True)

In [129]:
raw_all

,rent_hour,station_no,station_name,rent_type,sex,age_type,rent_count,momentum,green,distance,use_hour,address,gu
rent_date,,,,,,,,,,,,,
2021-01-01 00:00:00,0,1347,1347. 길음래미안아파트817동 상가 앞,정기권,,~10대,2.00,28.90,0.23,992.78,13.00,서울특별시 성북구 길음로 33,성북구
2021-01-01 00:00:00,0,1347,1347. 길음래미안아파트817동 상가 앞,정기권,,20대,1.00,11.24,0.07,283.72,1.00,서울특별시 성북구 길음로 33,성북구
2021-01-01 02:00:00,2,1347,1347. 길음래미안아파트817동 상가 앞,일일권,,20대,1.00,42.24,0.43,"1,839.04",37.00,서울특별시 성북구 길음로 33,성북구
2021-01-01 02:00:00,2,1347,1347. 길음래미안아파트817동 상가 앞,일일권,M,20대,2.00,50.75,0.44,"1,912.72",73.00,서울특별시 성북구 길음로 33,성북구
2021-01-01 05:00:00,5,1347,1347. 길음래미안아파트817동 상가 앞,일일권,M,~10대,1.00,183.65,1.66,"7,134.79",47.00,서울특별시 성북구 길음로 33,성북구
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-25 19:00:00,19,3878,3878. 장로회신학대학교 주기철 기념관,일일권,,20대,1.00,76.28,0.69,"2,963.57",53.00,NaN,NaN
2021-01-25 19:00:00,19,3878,3878. 장로회신학대학교 주기철 기념관,일일권,F,20대,2.00,105.18,0.99,"4,278.46",107.00,NaN,NaN
2021-01-27 13:00:00,13,3878,3878. 장로회신학대학교 주기철 기념관,정기권,,20대,1.00,73.45,0.51,"2,182.24",13.00,NaN,NaN


In [137]:
raw_all.isnull().sum()

rent_hour          0
station_no         0
station_name       0
rent_type          0
sex                0
age_type           0
rent_count         0
momentum           0
green              0
distance           0
use_hour           0
address         2533
gu              2533
dtype: int64

In [138]:
raw_all.index

DatetimeIndex(['2021-01-01 00:00:00', '2021-01-01 00:00:00',
               '2021-01-01 02:00:00', '2021-01-01 02:00:00',
               '2021-01-01 05:00:00', '2021-01-01 12:00:00',
               '2021-01-01 12:00:00', '2021-01-01 14:00:00',
               '2021-01-01 14:00:00', '2021-01-01 15:00:00',
               ...
               '2021-01-31 18:00:00', '2021-01-24 10:00:00',
               '2021-01-24 11:00:00', '2021-01-24 15:00:00',
               '2021-01-25 18:00:00', '2021-01-25 19:00:00',
               '2021-01-25 19:00:00', '2021-01-27 13:00:00',
               '2021-01-30 16:00:00', '2021-01-31 18:00:00'],
              dtype='datetime64[ns]', name='rent_date', length=720123, freq=None)

In [144]:
!pwd

/Users/johanjun/Documents/2021/seoul_bike_share/TSA Study


In [145]:
with open('/Users/johanjun/Documents/2021/seoul_bike_share/TSA Study/data/raw_all(datetime preprocessing).pkl', 'wb') as f:
    pickle.dump(raw_all, f)

In [165]:
df_1347 = raw_all[raw_all['station_no']==1347]

In [166]:
df_1347

,rent_hour,station_no,station_name,rent_type,sex,age_type,rent_count,momentum,green,distance,use_hour,address,gu
rent_date,,,,,,,,,,,,,
2021-01-01 00:00:00,0,1347,1347. 길음래미안아파트817동 상가 앞,정기권,,~10대,2.00,28.90,0.23,992.78,13.00,서울특별시 성북구 길음로 33,성북구
2021-01-01 00:00:00,0,1347,1347. 길음래미안아파트817동 상가 앞,정기권,,20대,1.00,11.24,0.07,283.72,1.00,서울특별시 성북구 길음로 33,성북구
2021-01-01 02:00:00,2,1347,1347. 길음래미안아파트817동 상가 앞,일일권,,20대,1.00,42.24,0.43,"1,839.04",37.00,서울특별시 성북구 길음로 33,성북구
2021-01-01 02:00:00,2,1347,1347. 길음래미안아파트817동 상가 앞,일일권,M,20대,2.00,50.75,0.44,"1,912.72",73.00,서울특별시 성북구 길음로 33,성북구
2021-01-01 05:00:00,5,1347,1347. 길음래미안아파트817동 상가 앞,일일권,M,~10대,1.00,183.65,1.66,"7,134.79",47.00,서울특별시 성북구 길음로 33,성북구
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-31 13:00:00,13,1347,1347. 길음래미안아파트817동 상가 앞,정기권,,40대,1.00,52.53,0.47,"2,040.88",13.00,서울특별시 성북구 길음로 33,성북구
2021-01-31 15:00:00,15,1347,1347. 길음래미안아파트817동 상가 앞,정기권,F,40대,1.00,170.73,1.54,"6,632.79",41.00,서울특별시 성북구 길음로 33,성북구
2021-01-31 15:00:00,15,1347,1347. 길음래미안아파트817동 상가 앞,일일권,M,30대,1.00,148.13,1.22,"5,268.69",36.00,서울특별시 성북구 길음로 33,성북구


In [171]:
df_1347 = pd.DataFrame(df_1347.groupby(df_1347.index)['rent_type'].agg('count'))

In [172]:
df_1347

,rent_type
rent_date,
2021-01-01 00:00:00,2
2021-01-01 02:00:00,2
2021-01-01 05:00:00,1
2021-01-01 12:00:00,2
2021-01-01 14:00:00,2
...,...
2021-01-31 11:00:00,1
2021-01-31 13:00:00,1
2021-01-31 15:00:00,2


In [173]:
df_1347.asfreq('H').isnull().sum()

rent_type    617
dtype: int64

In [174]:
df_1347.asfreq('H')[df_1347.asfreq('H').isnull().sum(axis=1) > 0]

,rent_type
rent_date,
2021-01-01 01:00:00,NaN
2021-01-01 03:00:00,NaN
2021-01-01 04:00:00,NaN
2021-01-01 06:00:00,NaN
2021-01-01 07:00:00,NaN
...,...
2021-01-31 14:00:00,NaN
2021-01-31 16:00:00,NaN
2021-01-31 17:00:00,NaN


In [176]:
df_1347 = df_1347.asfreq('H', method='ffill')
df_1347.isnull().sum()

rent_type    0
dtype: int64

In [177]:
df_1347.index

DatetimeIndex(['2021-01-01 00:00:00', '2021-01-01 01:00:00',
               '2021-01-01 02:00:00', '2021-01-01 03:00:00',
               '2021-01-01 04:00:00', '2021-01-01 05:00:00',
               '2021-01-01 06:00:00', '2021-01-01 07:00:00',
               '2021-01-01 08:00:00', '2021-01-01 09:00:00',
               ...
               '2021-01-31 12:00:00', '2021-01-31 13:00:00',
               '2021-01-31 14:00:00', '2021-01-31 15:00:00',
               '2021-01-31 16:00:00', '2021-01-31 17:00:00',
               '2021-01-31 18:00:00', '2021-01-31 19:00:00',
               '2021-01-31 20:00:00', '2021-01-31 21:00:00'],
              dtype='datetime64[ns]', name='rent_date', length=742, freq='H')

In [178]:
df_1347

,rent_type
rent_date,
2021-01-01 00:00:00,2
2021-01-01 01:00:00,2
2021-01-01 02:00:00,2
2021-01-01 03:00:00,2
2021-01-01 04:00:00,2
...,...
2021-01-31 17:00:00,2
2021-01-31 18:00:00,2
2021-01-31 19:00:00,2
